In [30]:
import numpy as np
import tensorflow as tf
from tf_agents.networks.q_network import QNetwork
import tf_agents.networks.network as network
from tf_agents.specs import tensor_spec
from snake_game import SnakeGame
from scene import Scene
from scene_longer_snake import SceneLongerSnake
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input

scene = SceneLongerSnake(init_randomly=True, snake_longer_prob=0.95, length_mean=10, length_std=2)
episodes_count = 80000
random_episodes = 20000


In [31]:
from tf_agents.environments.tf_py_environment import TFPyEnvironment

env = SnakeGame(scene)
env = TFPyEnvironment(env)


In [32]:
class MyQNetwork(network.Network):
    def __init__(self,
                 input_tensor_spec,
                 action_spec,
                 name='MyQNetwork'):
        super(MyQNetwork, self).__init__(
            input_tensor_spec=input_tensor_spec,
            state_spec=(),
            name=name)
        
        self._action_spec = action_spec
        matrix_shape = input_tensor_spec[0].shape
        obstacles_shape = input_tensor_spec[1].shape
        no_body_blocks_shape = input_tensor_spec[2].shape
        
        input0 = Input(shape=matrix_shape)
        input1 = Input(shape=obstacles_shape) # Each value represents if there is a wall or the snake body in the direction, 0 - no wall, 1 - wall
        input2 = Input(shape=no_body_blocks_shape) # Each value represents how many no body blocks are in the direction
        
        conv = Conv2D(32, (2, 2), 1, activation='relu', kernel_initializer='he_normal')(input0)
        flat = Flatten()(conv)
        dense1 = Dense(128, activation='relu', kernel_initializer='he_normal')(flat)
        dense2 = Dense(32, activation='relu', kernel_initializer='he_normal')(dense1)
        
        concat = tf.keras.layers.concatenate([dense2, input1, input2])
        output = Dense(4, activation='linear')(concat)
        
        self._model = tf.keras.Model(inputs=[input0, input1, input2], outputs=output)
        
        self._model.summary()

    def call(self, observations, step_type=None, network_state=(), training=False):
        output = self._model([observations[0], observations[1], observations[2]])
        return output, network_state

q_net = MyQNetwork(
    env.observation_spec(),
    env.action_spec())


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_20 (InputLayer)       [(None, 5, 5, 4)]            0         []                            
                                                                                                  
 conv2d_5 (Conv2D)           (None, 4, 4, 32)             544       ['input_20[0][0]']            
                                                                                                  
 flatten_5 (Flatten)         (None, 512)                  0         ['conv2d_5[0][0]']            


                                                                                                  
 dense_15 (Dense)            (None, 128)                  65664     ['flatten_5[0][0]']           
                                                                                                  
 dense_16 (Dense)            (None, 32)                   4128      ['dense_15[0][0]']            
                                                                                                  
 input_21 (InputLayer)       [(None, 4)]                  0         []                            
                                                                                                  
 input_22 (InputLayer)       [(None, 4)]                  0         []                            
                                                                                                  
 concatenate_5 (Concatenate  (None, 40)                   0         ['dense_16[0][0]',            
 )        

In [33]:
# Create the agent
from tf_agents.agents.dqn.dqn_agent import DdqnAgent
from tensorflow import keras
from keras.optimizers.legacy import Adam
from keras.losses import Huber
from tf_agents.trajectories import TimeStep

train_step_counter = tf.Variable(0)
optimizer = Adam(learning_rate=0.003)

epsilon = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=0.1,
    decay_steps=episodes_count,
    end_learning_rate=0.01
)

agent = DdqnAgent(
    time_step_spec=env.time_step_spec(),
    action_spec=env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=Huber(reduction="none"),
    gamma=tf.constant(0.95, dtype=tf.float32),
    train_step_counter=train_step_counter,
    epsilon_greedy=lambda: epsilon(train_step_counter)
)


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_23 (InputLayer)       [(None, 5, 5, 4)]            0         []                            
                                                                                                  
 conv2d_6 (Conv2D)           (None, 4, 4, 32)             544       ['input_23[0][0]']            
                                                                                                  
 flatten_6 (Flatten)         (None, 512)                  0         ['conv2d_6[0][0]']            
                                                                                                  
 dense_18 (Dense)            (None, 128)                  65664     ['flatten_6[0][0]']           
                                                                                            

In [34]:
# Create the replay buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size,
    max_length=100000
)


In [35]:
# Create the training metrics
from tf_agents.metrics import tf_metrics

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric()
]


In [36]:
# Create the driver
from tf_agents.drivers import dynamic_step_driver

collect_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=agent.collect_policy,
    observers=[replay_buffer.add_batch] + train_metrics,
    num_steps=2
)

# Run a random policy to fill the replay buffer
from tf_agents.policies.random_tf_policy import RandomTFPolicy

initial_collect_policy = RandomTFPolicy(env.time_step_spec(), env.action_spec())
init_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=initial_collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=random_episodes
)

final_time_step, final_policy_state = init_driver.run()


In [37]:
# Create the dataset
dataset = replay_buffer.as_dataset(
    sample_batch_size=64,
    num_steps=2, # Capaz cambiar esto
    num_parallel_calls=5
).prefetch(5)


In [38]:

# Create the training loop

from tf_agents.utils.common import function
from tf_agents.trajectories import trajectory
from tf_agents.eval.metric_utils import log_metrics
import logging

logging.getLogger().setLevel(logging.INFO)

collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

def train_agent(n_iterations):
    time_step = None
    policy_state = agent.collect_policy.get_initial_state(env.batch_size)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        if iteration % 1000 == 0:
            print("Iteration: ", iteration)
            print("Replay buffer len: " + str(replay_buffer.num_frames().numpy()))
            log_metrics(train_metrics)


In [39]:
train_agent(episodes_count)


INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 2
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


Iteration:  0
Replay buffer len: 43583


INFO:absl: 
		 NumberOfEpisodes = 255
		 EnvironmentSteps = 2002
		 AverageReturn = -9579.0
		 AverageEpisodeLength = 18.600000381469727


Iteration:  1000
Replay buffer len: 45838


INFO:absl: 
		 NumberOfEpisodes = 408
		 EnvironmentSteps = 4002
		 AverageReturn = -14873.5
		 AverageEpisodeLength = 15.899999618530273


...

Iteration:  77000
Replay buffer len: 100000


INFO:absl: 
		 NumberOfEpisodes = 7530
		 EnvironmentSteps = 156002
		 AverageReturn = -159941.5
		 AverageEpisodeLength = 37.70000076293945


Iteration:  78000
Replay buffer len: 100000


INFO:absl: 
		 NumberOfEpisodes = 7577
		 EnvironmentSteps = 158002
		 AverageReturn = -159753.5
		 AverageEpisodeLength = 30.399999618530273


Iteration:  79000
Replay buffer len: 100000


In [40]:
# Evaluate the agent

test_scene = Scene(init_randomly=True)
test_env = SnakeGame(test_scene)
test_env = TFPyEnvironment(test_env)

time_step = test_env.reset()
rewards = []
steps = 0

while not time_step.is_last() and steps < 200:
    steps += 1
    print("Map:\n", np.argmax(time_step.observation[0], axis=3))
    print("Direction:\n", time_step.observation[1])
    print("No body blocks:\n", time_step.observation[4])
    action_step = agent.policy.action(time_step)
    print("Action: ", action_step.action.numpy())
    time_step = test_env.step(action_step.action)
    rewards.append(time_step.reward.numpy()[0])
    
print("Total reward: ", sum(rewards))
print("Total steps: ", steps)


Map:
 [[[3 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 1]
  [0 0 0 0 2]]]
Direction:
 tf.Tensor([[False False  True  True]], shape=(1, 4), dtype=bool)
No body blocks:
 tf.Tensor([[20 15  0  4]], shape=(1, 4), dtype=uint8)
Action:  [0]
Map:
 [[[3 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 1 2]
  [0 0 0 0 0]]]
Direction:
 tf.Tensor([[False False  True False]], shape=(1, 4), dtype=bool)
No body blocks:
 tf.Tensor([[15 15  4  5]], shape=(1, 4), dtype=uint8)
Action:  [1]
Map:
 [[[3 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 1 0]
  [0 0 0 2 0]
  [0 0 0 0 0]]]
Direction:
 tf.Tensor([[False False False  True]], shape=(1, 4), dtype=bool)
No body blocks:
 tf.Tensor([[15 10  5  9]], shape=(1, 4), dtype=uint8)
Action:  [1]
Map:
 [[[3 0 0 0 0]
  [0 0 0 1 0]
  [0 0 0 2 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]
Direction:
 tf.Tensor([[False False False  True]], shape=(1, 4), dtype=bool)
No body blocks:
 tf.Tensor([[15  5  5 14]], shape=(1, 4), dtype=uint8)
Action:  [0]
Map:
 [[[3 0 0 0 0]
  [0 0 1 2 0]
  [0 0 0 0

In [41]:

# Count the average steps per episode

n = 500
total_steps = 0
total_reward = 0
steps = 0
total_500 = 0
wins = 0

for i in range(n):
    if i % 10 == 0 and i != 0:
        print("Episode: ", i)
        
    time_step = test_env.reset()
    steps = 0
    episode_reward = 0
    while not time_step.is_last() and steps < 500:
        steps += 1
        action_step = agent.policy.action(time_step)
        time_step = test_env.step(action_step.action)
        reward = time_step.reward.numpy()[0]
        total_reward += reward
        episode_reward += reward
        
    if episode_reward == 115:
        wins += 1
    
    if steps == 500:
        total_500 += 1
    else:
        total_steps += steps
        
    if i % 10 == 0 and i != 0:
        print("Avg reward: ", total_reward / (i + 1))
        
if steps == 500:
    print("500 episodes reached")
    
print("Average steps per episode: ", total_steps / (n - total_500))
print("Average reward per episode: ", total_reward / n)
print("Total 500 episodes: ", total_500)
print("Wins: ", wins)


Episode:  10
Avg reward:  54.54545454545455
Episode:  20
Avg reward:  53.80952380952381
Episode:  30
Avg reward:  44.193548387096776
Episode:  40
Avg reward:  41.34146341463415
Episode:  50
Avg reward:  42.05882352941177
Episode:  60
Avg reward:  43.032786885245905
Episode:  70
Avg reward:  42.11267605633803
Episode:  80
Avg reward:  41.97530864197531
Episode:  90
Avg reward:  43.79120879120879
Episode:  100
Avg reward:  44.10891089108911
Episode:  110
Avg reward:  43.42342342342342
Episode:  120
Avg reward:  42.93388429752066
Episode:  130
Avg reward:  43.93129770992366
Episode:  140
Avg reward:  45.283687943262414
Episode:  150
Avg reward:  44.70198675496689
Episode:  160
Avg reward:  44.78260869565217
Episode:  170
Avg reward:  44.35672514619883
Episode:  180
Avg reward:  43.97790055248619
Episode:  190
Avg reward:  43.7434554973822
Episode:  200
Avg reward:  43.85572139303483
Episode:  210
Avg reward:  43.62559241706161
Episode:  220
Avg reward:  43.665158371040725
Episode:  230
Av

In [42]:
# Save the q network weights
tf.keras.models.save_model(q_net._model, "q_network.h5")

/var/folders/bw/_7s8wxw93cngpt5f7bg5s3hm0000gn/T/ipykernel_26610/1164216016.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(q_net._model, "q_network.h5")
